This notebook is done following 
* [Building text classifier with Differential Privacy](https://github.com/pytorch/opacus/blob/main/tutorials/building_text_classifier.ipynb)
* [Fine-tuning with custom datasets](https://huggingface.co/transformers/v3.4.0/custom_datasets.html#seq-imdb)

# Intial Setup
https://huggingface.co/docs/transformers/training

## Install

In [ ]:
!pip install opacus
# !pip install transformers
!pip install datasets
import datasets

## Import

In [ ]:
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification
from torch.optim import AdamW
import torch
from torch.utils.data import DataLoader

from opacus.utils.batch_memory_manager import BatchMemoryManager

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import gc

pd.set_option('display.max_columns', None)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Config

In [ ]:
from dataclasses import dataclass

@dataclass
class Config:
    # train config
    model_name = 'bert-base-uncased'
    batch_size = 64
    learning_rate = 1e-4
    epochs = 20
    num_labels = 2

    dataset_name = 'social_bias_frames'
    text_column = 'post'
    # if id column is string, replace that with an integer index during preprocessing
    raw_id_column = 'HITId'
    id_column = 'index'

    # the original id column HITId has been replaced with index because it was string 
    # and torch didn't support str format
    raw_id_column = 'HITId'
    id_column = 'index'

    # Private training config
    delta_list = [5e-2, 1e-3, 1e-5]
    noise_multiplier = 0.4
    max_grad_norm = 1
    max_physical_batch_size = 32

    # target in raw dataset. However, it will be renamed to `labels` here to facilitate training setup
    raw_target_column = 'offensiveYN'
    target_column = 'labels'

    # If needs to be splitted into train test validation set
    need_to_split = False
    # test and validation data with each be 50% of this amount
    test_size = 0.3
    max_seq_length = 128
    seed = 2022

## Set seed

In [ ]:
import random

def seed_torch(seed=7):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    

global_seed = Config.seed
seed_torch(global_seed)

## Get device

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

# Load tokenized data

From my [other notebook](https://www.kaggle.com/code/khairulislam/tokenize-jigsaw-comments). The dataset is tokenized from the [Jigsaw competition]( https://www.kaggle.com/competitions/jigsaw-unintended-bias-in-toxicity-classification) and [all_data.csv](https://www.kaggle.com/competitions/jigsaw-unintended-bias-in-toxicity-classification/data?select=all_data.csv)

In [ ]:
text = Config.text_column
target = Config.target_column
root = '/kaggle/input/tokenize-social-bias-using-bert/'

In [ ]:
import pickle
    
with open(root + 'train.pkl', 'rb') as input_file:
    train_tokenized = pickle.load(input_file)
    input_file.close()
    
with open(root + 'validation.pkl', 'rb') as input_file:
    validation_tokenized = pickle.load(input_file)
    input_file.close()
    
with open(root + 'test.pkl', 'rb') as input_file:
    test_tokenized = pickle.load(input_file)
    input_file.close()

In [ ]:
print(train_tokenized)

In [ ]:
# Remove id column from the data to be batched
id_column = Config.id_column 
train_tokenized = train_tokenized.remove_columns(id_column)
test_tokenized = test_tokenized.remove_columns(id_column)
validation_tokenized = validation_tokenized.remove_columns(id_column)

# Private Training

## Data loader

[How to choose batch size in DP](https://github.com/pytorch/opacus/blob/main/tutorials/building_text_classifier.ipynb)

In [ ]:
BATCH_SIZE = Config.batch_size

train_dataloader = DataLoader(train_tokenized, batch_size=BATCH_SIZE)
validation_dataloader = DataLoader(validation_tokenized, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_tokenized, batch_size=BATCH_SIZE)

## Model and optimizer

In [ ]:
# add the utility script from File->Add utility script
from train_utils import TrainUtil, ModelCheckPoint, EarlyStopping

num_labels = Config.num_labels
model_name = Config.model_name
train_util = TrainUtil(Config.id_column, Config.target_column, device)

In [ ]:
# load a fresh model each time
model = TrainUtil.load_pretrained_model(model_name, num_labels)

# Set the model to train mode (HuggingFace models load in eval mode)
model = model.train().to(device)
LEARNING_RATE = Config.learning_rate
# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)
EPOCHS = Config.epochs

# https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ReduceLROnPlateau.html
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, verbose=True)

In [ ]:
model_type = model_name.split(r'/')[-1]
result_dir = '' # f'dp models/{model_type}/{NOISE_MULTIPLIER*10}/'
best_model_path = os.path.join(result_dir, 'model.pt')

if result_dir != '':
    os.makedirs(result_dir, exist_ok=True)

check_point = ModelCheckPoint(filepath=best_model_path)
early_stopping = EarlyStopping(patience=3, min_delta=0)

## Privacy Engine

In [ ]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()

In [ ]:
model, optimizer, train_dataloader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    noise_multiplier=Config.noise_multiplier,
    max_grad_norm=Config.max_grad_norm,
    poisson_sampling=False,
)

## Loop

In [ ]:
start_epoch = 1
# load a previous model if there is any
# model, optimizer, lr_scheduler, start_epoch = load_model(model, optimizer, lr_scheduler, device, filepath=best_model_path)

for epoch in range(start_epoch, EPOCHS+1):
    gc.collect()
    
    with BatchMemoryManager(
        data_loader=train_dataloader, 
        max_physical_batch_size=Config.max_physical_batch_size, 
        optimizer=optimizer
    ) as memory_safe_data_loader:
        train_loss, train_result, train_probs = train_util.dp_train(
            model, optimizer, epoch, memory_safe_data_loader
        )
    val_loss, val_result, val_probs = train_util.evaluate(
        model, validation_dataloader, epoch, 'Validation'
    )

    epsilons = []
    for delta in Config.delta_list:
        epsilons.append(privacy_engine.get_epsilon(delta))

    print(
      f"Epoch: {epoch} | "
      f"ɛ: {np.round(epsilons, 2)} |"
      f"Train loss: {train_loss:.3f} | "
      f"Train result: {train_result} |\n"
      f"Validation loss: {val_loss:.3f} | "
      f"Validation result: {val_result} | "
    )
    
    loss = -val_result['f1']
    lr_scheduler.step(loss)
    check_point(model, optimizer, lr_scheduler, epoch, loss)
    
    early_stopping(loss)
    if early_stopping.early_stop:
        break
    print()
    # break

In [ ]:
# load the best model
model, _, _, best_epoch = TrainUtil.load_model(model, optimizer, lr_scheduler, device, filepath=best_model_path)

train_loss, train_result, train_probs = train_util.evaluate(model, train_dataloader, best_epoch, 'Train')
# no need to reevaluate if the validation set if the last model is the best one
if best_epoch != epoch:
    val_loss, val_result, val_probs = train_util.evaluate(model, validation_dataloader, best_epoch, 'Validation')
test_loss, test_result, test_probs = train_util.evaluate(model, test_dataloader, best_epoch, 'Test')

## Dump results and others

In [ ]:
# load the original tokenized files, since we removed the id columns earlier
# and id columns are needed for the result dumping part
with open(root + 'train.pkl', 'rb') as input_file:
    train_tokenized = pickle.load(input_file)
    input_file.close()
    
with open(root + 'validation.pkl', 'rb') as input_file:
    validation_tokenized = pickle.load(input_file)
    input_file.close()
    
with open(root + 'test.pkl', 'rb') as input_file:
    test_tokenized = pickle.load(input_file)
    input_file.close()

In [ ]:
# Save the results
train_util.dump_results(
    result_dir,train_probs, train_tokenized, 
    val_probs, validation_tokenized, test_probs, test_tokenized
)

## Save config

In [ ]:
import json

config_dict = dict(Config.__dict__)
# exclude hidden variables
keys = list(config_dict.keys())
for key in keys:
    if key.startswith('__'):
        del config_dict[key]
        
with open(os.path.join(result_dir, 'config.json'), 'w') as output:
    json.dump(config_dict, output, indent=4)